In [1]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [2]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'factory automation' # topic is factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(10) # first 10 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/ferdinandk/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 10 documents.


In [4]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


Aniai, a New York City-based robotic company specializing in kitchen solutions, recently secured $12 million in Pre-Series A funding. The funding round was led by InterVest, with contributions from SV Investment, UK-based Ignite Innovation, and existing investor Capstone Partners. The company plans to utilize the funds to establish a dedicated manufacturing facility known as Factory One in South Korea. This move aims to ensure stable production and efficient supply chain management for its flagship product, the Alpha Grill, while supporting the company's expansion efforts in North America and globally.

Founded in 2020 by CEO Gunpil Hwang, Aniai focuses on developing robotic solutions for cooking hamburgers. The Alpha Grill, their primary product, uses robotics to cook hamburgers with a double-sided grill capable of cooking up to eight patties simultaneously or 200 patties per hour. By automating tasks such as pressing and flipping patties, the Alpha Grill streamlines restaurant operat

Numbers Station, a company based in Menlo Park, California, has successfully raised a total of $17.5 million in funding through a series A round and seed funding. The series A funding round, totaling $12.5 million, was led by Madrona, a prominent investment firm. Other participants in this round included Norwest Venture Partners, Factory, and notable angel investors like Jeff Hammerbacher, who is known for co-founding Cloudera. This recent funding round follows a previous seed round that raised $5 million. In total, Numbers Station has now amassed $17.5 million in capital.

Numbers Station distinguishes itself as an intelligent data stack automation platform that leverages foundation model technology originally developed at the Stanford AI lab. This technology empowers data workers by automating laborious data tasks, initially focusing on data transformation work. The platform enables users to swiftly extract valuable insights from data, streamlining the process and enhancing efficienc

The text discusses the recent success of Software Defined Automation in securing $10 million in seed funding. The funding round was led by Insight Partners, with participation from Baukunst VC, Fly Ventures, and First Momentum. Following the investment, Jon Rosenbaum from Insight Partners and Axel Bichara from Baukunst will be joining Software Defined Automation's Board of Directors.

Software Defined Automation specializes in revolutionizing factory automation through its Industrial-Control-as-a-Service (ICaaS) offering. This approach involves cloud-based management of existing Programmable Logic Controllers (PLCs) under TechOps, Git-enabled PLC code versioning and collaboration in DevOps, and virtualization of PLCs on edge servers through Virtual PLC. By integrating these technologies, Software Defined Automation aims to dismantle proprietary silos in control technology stacks, facilitating an API-driven modern microservices architecture.

The company operates from two locations, Bos

In [5]:
Problem = """Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry."""

print(Problem)

Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry.


# Answering questions on the results with OpenAI GPT

In [12]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt = f""" You are Senior Vice President Technology and Innovation & CTO. You were just handed the following problem:

       
        {Problem}
        
     When you evaluate a new business idea, you strictly adhere to the following guiding principles:
        - Prioritize quick, low-investment idea testing.
        - Aim for large profits with smaller budgets, avoiding investments over 10-15 million per portfolio element.
        - Target a minimum 10 million EUR annual revenue.
        - Specialize in LLMs in Engineering and Operations.
        - Avoid new ecosystem creation.
        - Prioritize product over solution business for scalability.
        - Consider AI as a potential product.
        - Ensure all initiatives align with business profitability.
        - Aim for non-cannibalizing, fringe markets.

        Your work is highly crucial and you are responsible of coming up with the next best innovation.
        """

budgets = [1000, 10000, 100000,1000000,10000000]  # Budget constraints for each solution proposal
summary_solutions = []

user_prompt = f"""
        As an input, you are give the {Problem}.
        For each budget USD {budgets}, come up with one different solution, each tailored to this specific budget. 
        The solutions should be innovative and be achievable within the budget.
        For each budget, you should leverage insights or approaches mentioned in the following document summary:

        Document summary:
        ```
        {summary_list}
        ```

        Please make sure you use the budgets that were set correctly, and that the idea is really achievable within that budget.
      
     
        Please summarize in one paragraph for each solution, with the following content:
    
        “Name”: Solution name (5 words max)
        “Solved Problem”: 1 sentence summary of the problem that is being solved and how much budget is needed
        “Solution”: Provide a short summary of the solution and how it works (4 to 5 sentences max)
        “Impact”: How does the solution solve the specific problem (3 sentences max)
        “Technology”: Please also give a very detailed explanation of which technology is utilized and how it is utilized. Please be very diligent in this task, imagine you are explaining it to the CTO who has a background of engineering and has to implement the solution  (4 to 5 sentences max)
        “Sources”: Please provide the specific sources of inspiration that you used from the {summary_list} (1 sentence max)
    

    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions.append(completion.choices[0].message.content)


### Solution for Budget USD 1,000: 
**Name:** Basic Data Sync Tool  
**Solved Problem:** For a budget of USD 1,000, we address the issue of manual data synchronization across different automation systems.  
**Solution:** The solution involves the development of a basic software script that facilitates the initial manual synchronization of data between different factory automation systems. This script will use existing APIs or file transfer mechanisms to exchange data, providing a rudimentary form of integration and reducing instances of manual entry.  
**Impact:** Although basic, this solution will reduce the time spent on manual data entry and errors associated with it, thus slightly improving workflow efficiency and data accuracy in small-scale environments.  
**Technology:** The technology utilized here is primarily scripting languages such as Python or JavaScript. These languages are chosen for their wide support for APIs and ease of handling data formats like JSON or XML, enabling

In [13]:
# Initial rating of the solutions

initial_rating = []

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """

user_prompt_2 = f""""

Please take all solutions from the following text: {summary_solutions}
First, repeat the following points for the first solution:
    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”

Then, rate the first solution according to the following criteria. Please justify your ratings in max one sentence. Please be very critical in your ratings and only give a 3 out of 3 rating when it is very good:   

    - Functionality and Performance: Assess whether the solution addresses and solves the problem it’s supposed to solve. This includes estimating if the solution can solve the problem quickly and effectively. (1 = low performance, 2 = medium performance, 3 = high performance)
    - Scalability: Evaluate whether the solution can adapt to future growth (or shrinking) of demand and accommodate changes in requirements: How flexible is the solution? (1 = low scalability, 2 = medium scalability, 3 = high scalability)
    - Cost-effectiveness: Assess the costs associated with the solution. How large are upfront costs for implementation such as training, facilities, machines, etc.? How large are operating costs such as personnel, energy, maintenance, etc.? The solution should have a relatively high return on investment. (1 = low cost-effectiveness, 2 = medium cost-effectiveness 3 = high cost-effectiveness)
    - Innovativeness: How innovative is the solution? Do similar solutions for the problem exist already? (1 = low innovativeness, 2 = medium innovativeness 3 = high innovativeness)
    - Uniqueness: The solutions contained in the list should differ substantially and not be similar. (1 = low uniqueness, 2 = medium uniqueness 3 = high uniqueness)
    - Overall Rating: Sum of all the previous ratings, out of 15

Now, repeat these steps for all solutions.

"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt_2}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating.append(completion.choices[0].message.content)



### Solution for Budget USD 1,000: 
- **Name:** Basic Data Sync Tool  
- **Solved Problem:** For a budget of USD 1,000, we address the issue of manual data synchronization across different automation systems.  
- **Solution:** The solution involves the development of a basic software script that facilitates the initial manual synchronization of data between different factory automation systems. This script will use existing APIs or file transfer mechanisms to exchange data, providing a rudimentary form of integration and reducing instances of manual entry.  
- **Impact:** Although basic, this solution will reduce the time spent on manual data entry and errors associated with it, thus slightly improving workflow efficiency and data accuracy in small-scale environments.  
- **Technology:** The technology utilized here is primarily scripting languages such as Python or JavaScript. These languages are chosen for their wide support for APIs and ease of handling data formats like JSON or XML

In [ ]:
user_prompt_3 = f""""

Please take all the solutions and the corresponding ratings from: {initial_rating}

Take the three solutions with the highest overall rating out of 15, sorting them in descending rating order.

Outline the following points for these solution:
    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”


"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt_3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability